In [1]:
import stanza

In [2]:
import torch

In [3]:
torch.cuda.is_available()

False

In [ ]:
!ls -all ../saved_models/ner

In [4]:
stanza.download('ru')

2020-06-06 11:55:48 INFO: Downloading default packages for language: ru (Russian)...
2020-06-06 11:55:49 INFO: File exists: /home/captain/stanza_resources/ru/default.zip.
2020-06-06 11:55:54 INFO: Finished downloading models and saved to /home/captain/stanza_resources.


In [5]:
nlp = stanza.Pipeline('ru', processors='tokenize,ner', tokenize_pretokenized=True, ner_model_path="../saved_models/ner/ru_pampers_nertagger.pt")

2020-06-06 11:55:54 INFO: Loading these models for language: ru (Russian):
| Processor | Package                 |
---------------------------------------
| tokenize  | syntagrus               |
| ner       | ../saved_m...rtagger.pt |

2020-06-06 11:55:54 INFO: Use device: cpu
2020-06-06 11:55:54 INFO: Loading: tokenize
2020-06-06 11:55:54 INFO: Loading: ner
2020-06-06 11:55:55 INFO: Done loading processors!


In [ ]:
#nlp.loaded_processors[1].pipeline.processors["ner"].config

In [6]:
print(nlp.loaded_processors[1].pipeline.processors["tokenize"].config)

{'model_path': '/home/captain/stanza_resources/ru/tokenize/syntagrus.pt', 'pretokenized': True, 'lang': 'ru', 'mode': 'predict'}


In [7]:
doc = nlp("Подгузники-трусики Bella baby Happy")
print(*[f'entity: {ent.text}\ttype: {ent.type}' for sent in doc.sentences for ent in sent.ents], sep='\n')

entity: Подгузники-трусики	type: PRODUCT
entity: Bella	type: ORG
entity: baby	type: PRODUCT
entity: Happy	type: PRODUCT


In [8]:
from stanza.models.ner.data import DataLoader

In [9]:
doc = nlp("Подгузники-трусики Bella baby Happy")
print(*[f'entity: {ent.text}\ttype: {ent.type}' for sent in doc.sentences for ent in sent.ents], sep='\n')

entity: Подгузники-трусики	type: PRODUCT
entity: Bella	type: ORG
entity: baby	type: PRODUCT
entity: Happy	type: PRODUCT


In [11]:
doc = nlp("Подгузники-трусики Bella baby Happy")
print(*[f'entity: {ent.text}\ttype: {ent.type}' for sent in doc.sentences for ent in sent.ents], sep='\n')

entity: Подгузники-трусики	type: PRODUCT
entity: Bella	type: ORG
entity: baby	type: PRODUCT
entity: Happy	type: PRODUCT


In [12]:
doc = nlp("Трусики-подгузники PAMPERS 2-3кг 28шт")
print(*[f'entity: {ent.text}\ttype: {ent.type}' for sent in doc.sentences for ent in sent.ents], sep='\n')

entity: Трусики-подгузники	type: PRODUCT
entity: PAMPERS	type: ORG
entity: 2-3кг	type: QUANTITY
entity: 28шт	type: QUANTITY


In [13]:
def predict(line):
    doc = nlp(line)
    labels = [ent.type for sent in doc.sentences for ent in sent.ents]
    text = [ent.text for sent in doc.sentences for ent in sent.ents]
    return (labels, text)
predict("Влажные салфетки Bella Sensitive, детские, 208 шт")

(['PRODUCT', 'PRODUCT', 'ORG', 'PRODUCT', 'PRODUCT', 'QUANTITY', 'QUANTITY'],
 ['Влажные', 'салфетки', 'Bella', 'Sensitive,', 'детские,', '208', 'шт'])

 ### получить данные как остальные классификаторы

In [14]:
from collections import Counter

counter_TP= Counter()
counter_FN = Counter()
counter_FP = Counter()

def calculate_match(true_values, pred_values):
    for i, label in enumerate(filter(lambda v: v !="O",true_values)):
        if len(pred_values) <= i:
            counter_FN[label] += 1
            #counter_FP[pred_value] +=1
            continue
            
        pred_value = pred_values[i]    
        if  label == pred_value:
            counter_TP[label] += 1
        else:
            counter_FN[label] += 1
            counter_FP[pred_value] +=1

### прочитать валидационный корпус

In [15]:
valid_df_path="../data/ner/Russian-pampers/dev.bio"
test_dataset = []
with open(valid_df_path, "r") as fp:
    acc = []
    for line in fp:
        if (line != "\n"):
            tmp = line.strip().split("\t")
            acc.append((tmp[0], tmp[1]))
        else:
            test_dataset.append(acc)
            acc = []
test_dataset[:2]

[[('text', 'labels'),
  ('Подгузники', 'PRODUCT'),
  ('Pampers', 'ORG'),
  ('Procare', 'PRODUCT'),
  ('1', 'ORDINAL'),
  ('2-5кг', 'QUANTITY'),
  ('38шт', 'QUANTITY')],
 [('Подгузники', 'PRODUCT'),
  ('Pampers', 'ORG'),
  ('Procare', 'PRODUCT'),
  ('2', 'ORDINAL'),
  ('3-6кг', 'QUANTITY'),
  ('36шт', 'QUANTITY')]]

In [16]:
test_labels = []
test_texts = []
for line in test_dataset:
    test_labels.append(list(map(lambda x: x[1], line)))
    test_texts.append(" ".join(map(lambda x: x[0], line)))
    
print(test_labels[:2])
print(test_texts[:2])

[['labels', 'PRODUCT', 'ORG', 'PRODUCT', 'ORDINAL', 'QUANTITY', 'QUANTITY'], ['PRODUCT', 'ORG', 'PRODUCT', 'ORDINAL', 'QUANTITY', 'QUANTITY']]
['text Подгузники Pampers Procare 1 2-5кг 38шт', 'Подгузники Pampers Procare 2 3-6кг 36шт']


### получить предсказания

In [17]:
%%time
pred_labels = []
pred_text =[]
for line in test_texts:
    tmp = predict(line)
    pred_labels.append(tmp[0])
    pred_text.append(tmp[1])
    
len([item for sublist in pred_labels for item in sublist])

CPU times: user 4.65 s, sys: 1.92 ms, total: 4.65 s
Wall time: 783 ms


426

In [18]:
counter_TP= Counter()
counter_FN = Counter()
counter_FP = Counter()

for y_test_l, y_pred_l in zip(test_labels, pred_labels):
    calculate_match(y_test_l, y_pred_l)
    
labels=["ORDINAL", "ORG", "PRODUCT", "QUANTITY"]
print("Label\tPrecision\tRecall\tF1\tTruePositive\tFalsePositiv\tFalseNegative")
for label in labels:
    precision = counter_TP.get(label, 0) / (counter_TP.get(label, 1) + counter_FP.get(label, 0))
    recall = counter_TP.get(label, 0) / (counter_TP.get(label, 1) + counter_FN.get(label, 0))
    f1 = 2 * (precision*recall) / max((precision + recall),1)
    print("%s\t\t%.3f\t\t%.3f\t%.3f\t%d\t\t%d\t\t%d" % (label, precision, recall, f1, counter_TP.get(label, 0), counter_FP.get(label, 0), counter_FN.get(label, 0)))

Label	Precision	Recall	F1	TruePositive	FalsePositiv	FalseNegative
ORDINAL		1.000		0.944	0.971	51		0		3
ORG		1.000		1.000	1.000	53		0		0
PRODUCT		1.000		1.000	1.000	171		0		0
QUANTITY		0.980		0.987	0.983	147		3		2


In [19]:
def calculate_match_debug(true_values, pred_values):
    for i, label in enumerate(filter(lambda v: v !="O",true_values)): #stamza не показывает O
        if len(pred_values) <= i:
            counter_FN[label] += 1
            #counter_FP[pred_value] +=1
            continue
            
        pred_value = pred_values[i]    
        if  label == pred_value:
            print (f"{i}:{label} == {pred_value}")
            counter_TP[label] += 1
        else:
            print(f"{i}:{label} <> {pred_value}")
            counter_FN[label] += 1
            counter_FP[pred_value] +=1

In [20]:
counter_TP= Counter()
counter_FN = Counter()
counter_FP = Counter()

index = 20 #11, 15
print(test_dataset[index])
print(list(zip(pred_text[index], pred_labels[index])))
print("-------------------------------------------------------------------------")
calculate_match_debug(test_labels[index], pred_labels[index])
labels=["ORDINAL", "ORG", "PRODUCT", "QUANTITY"]
print("-------------------------------------------------------------------------")
print("Label\t\tPrecision\tRecall\tF1\tTruePositive\tFalsePositiv\tFalseNegative")
for label in labels:
    precision = counter_TP.get(label, 0) / (counter_TP.get(label, 1) + counter_FP.get(label, 0))
    recall = counter_TP.get(label, 0) / (counter_TP.get(label, 1) + counter_FN.get(label, 0))
    f1 = 2 * (precision*recall) / max((precision + recall),1)
    print("%s\t\t%.3f\t\t%.3f\t%.3f\t%d\t\t%d\t\t%d" % (label, precision, recall, f1, counter_TP.get(label, 0), counter_FP.get(label, 0), counter_FN.get(label, 0)))

[('Подгузники', 'PRODUCT'), ('Pampers', 'ORG'), ('Premium', 'PRODUCT'), ('Care', 'PRODUCT'), ('Midi', 'PRODUCT'), ('3', 'ORDINAL'), ('6-10кг', 'QUANTITY'), ('52шт', 'QUANTITY')]
[('Подгузники', 'PRODUCT'), ('Pampers', 'ORG'), ('Premium', 'PRODUCT'), ('Care', 'PRODUCT'), ('Midi', 'PRODUCT'), ('3', 'ORDINAL'), ('6-10кг', 'QUANTITY'), ('52шт', 'QUANTITY')]
-------------------------------------------------------------------------
0:PRODUCT == PRODUCT
1:ORG == ORG
2:PRODUCT == PRODUCT
3:PRODUCT == PRODUCT
4:PRODUCT == PRODUCT
5:ORDINAL == ORDINAL
6:QUANTITY == QUANTITY
7:QUANTITY == QUANTITY
-------------------------------------------------------------------------
Label		Precision	Recall	F1	TruePositive	FalsePositiv	FalseNegative
ORDINAL		1.000		1.000	1.000	1		0		0
ORG		1.000		1.000	1.000	1		0		0
PRODUCT		1.000		1.000	1.000	4		0		0
QUANTITY		1.000		1.000	1.000	2		0		0
